# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time 
%matplotlib inline

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
#Load the data
weather_data_df = pd.read_csv(r"C:\Users\kndrs\Python-Api-Challenge\Weather Py\cities.csv")
weather_data_df


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Jamestown,90,US,1619316103,50,42.0970,-79.2353,52.00,9.22
1,Ambon City,40,ID,1619315855,62,-3.6954,128.1814,89.60,5.75
2,George Town,20,MY,1619316130,89,5.4112,100.3354,84.20,4.61
3,Mariana,0,BR,1619316130,92,-20.3778,-43.4161,62.58,1.83
4,Busselton,1,AU,1619316130,69,-33.6500,115.3333,69.22,1.12
...,...,...,...,...,...,...,...,...,...
538,Lorengau,13,PG,1619316325,71,-2.0226,147.2712,83.57,5.32
539,Puerto del Rosario,0,ES,1619316185,64,28.5004,-13.8627,68.00,5.75
540,Karachi,0,PK,1619316072,57,24.9056,67.0822,78.80,7.00
541,Nuuk,75,GL,1619316326,49,64.1835,-51.7216,46.40,12.66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data_df[["Latitude", "Longitude"]]

# Store Humidity in humidity
humidity = weather_data_df["Humidity"]

In [19]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
print(fig)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
narrow_city_df = weather_data_df.loc[(weather_data_df["Wind Speed"] <= 10) & (weather_data_df["Cloudiness"] == 0) & 
                                   (weather_data_df["Max Temp"] >= 70) & (weather_data_df["Max Temp"] <= 80)].dropna()

narrow_city_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
62,Atar,0,MR,1619316151,16,20.5169,-13.0499,77.70,4.97
96,Kozan,0,TR,1619316163,55,37.4552,35.8157,72.00,2.30
142,Carnarvon,0,AU,1619316180,43,-24.8667,113.6333,75.20,8.05
307,Martinópolis,0,BR,1619316148,49,-22.1458,-51.1708,71.60,4.61
378,Port Macquarie,0,AU,1619316267,54,-31.4333,152.9167,73.00,1.99
379,Perth,0,AU,1619316267,56,-31.9333,115.8333,75.99,2.30
414,Caravelas,0,BR,1619316280,89,-17.7125,-39.2481,73.40,4.81
457,Buraidah,0,SA,1619316296,28,26.3260,43.9750,73.40,5.23
458,Khadro,0,PK,1619316297,60,26.1472,68.7176,75.20,4.61
486,São Mateus,0,BR,1619316237,94,-18.7161,-39.8589,71.37,2.15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:

# Create a hotel_df
hotel_df = narrow_city_df.loc[:,["City","Country", "Latitude", "Longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
62,Atar,MR,20.5169,-13.0499,
96,Kozan,TR,37.4552,35.8157,
142,Carnarvon,AU,-24.8667,113.6333,
307,Martinópolis,BR,-22.1458,-51.1708,
378,Port Macquarie,AU,-31.4333,152.9167,
379,Perth,AU,-31.9333,115.8333,
414,Caravelas,BR,-17.7125,-39.2481,
457,Buraidah,SA,26.3260,43.9750,
458,Khadro,PK,26.1472,68.7176,
486,São Mateus,BR,-18.7161,-39.8589,


In [23]:


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 62: Atar.
Closest hotel in Atar is Bab Sahara.
------------
Retrieving Results for Index 96: Kozan.
Closest hotel in Kozan is Avşaroğlu Otel.
------------
Retrieving Results for Index 142: Carnarvon.
Closest hotel in Carnarvon is Hospitality Carnarvon.
------------
Retrieving Results for Index 307: Martinópolis.
Closest hotel in Martinópolis is Hotel Ouro Verde.
------------
Retrieving Results for Index 378: Port Macquarie.
Closest hotel in Port Macquarie is ibis Styles Port Macquarie.
------------
Retrieving Results for Index 379: Perth.
Closest hotel in Perth is Peppers Kings Square Perth.
------------
Retrieving Results for Index 414: Caravelas.
Closest hotel in Caravelas is Mar Doce de Abrolhos.
------------
Retrieving Results for Index 457: Buraidah.
Closest hotel in Buraidah is Mövenpick Hotel Qassim.
------------
Retrieving Results for Index 458: Khadro.
Closest hotel in Khadro is Amjad Bhatti Hotel.
------------
Retrieving Results for Index 486: São

In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [25]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))